In [1]:
import torch
import esm

from lib.data.datasets.GB1 import get_GB1_dataset
from lib.utils.file import save_pt_file

In [2]:
# checkpoints are downloaded to "C:\Users\Matouš/.cache\torch\hub\checkpoints\"
#esm2_t6_8M_UR50D #esm2_t33_650M_UR50D #esm1b_t33_650M_UR50S
model, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()

batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [3]:
sequences, fitness, variants = get_GB1_dataset(
    return_variants=True,
)

s:\Documents\master\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [4]:
data = []
for s in range(len(sequences)):
    data.append((variants[s], sequences[s]))
variants, sequences, tokenized_sequences = batch_converter(data)

tokenized_sequences = tokenized_sequences.to(device)

In [5]:
# Extract per-residue representations
SEQUENCE_LENGTH = len(tokenized_sequences[0])
EMBEDDING_LENGTH = 1280

embeddings = torch.empty(
    (len(tokenized_sequences), EMBEDDING_LENGTH), dtype=torch.float32
)
with torch.no_grad():
    for s in range(len(tokenized_sequences)):
        results = model(
            tokenized_sequences[s : s + 1, :], repr_layers=[33], return_contacts=True
        )
        representation = results["representations"][33]
        representation = representation[0,1:SEQUENCE_LENGTH-1].mean(0)
        embeddings[s, :] = representation

embeddings.size()

torch.Size([149361, 1280])

In [6]:
save_pt_file(
    embeddings,
    save_to="./../../data/GB1/esm-1b_embedding.pt",
    absolute_path=False,
)

saving to file "s:\Documents\master\code\llm./../../data/GB1/esm-1b_embedding.pt"


In [7]:
save_pt_file(
    tokenized_sequences,
    save_to="./../../data/GB1/esm-1b_sequences.pt",
    absolute_path=False,
)

saving to file "s:\Documents\master\code\llm./../../data/GB1/esm-1b_sequences.pt"


In [9]:
import pandas as pd

fitness_norm = torch.load("./../../data/GB1/progen2_fitness_norm.pt")

data = {
    "Variants": variants,
    "Fitness": fitness,
    "Fitness_norm": fitness_norm,
    "Sequences": [seq for seq in tokenized_sequences.to("cpu")],
    "Embedding": [emb for emb in embeddings.to("cpu")],
}
df = pd.DataFrame(data)

In [10]:
save_pt_file(
    df,
    save_to="./../../data/GB1/esm-1b_dataframe.pt",
    absolute_path=False,
)

saving to file "s:\Documents\master\code\llm./../../data/GB1/esm-1b_dataframe.pt"
